<a href="https://colab.research.google.com/github/Dazhou2000/Msc-Project/blob/master/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score
import optuna
import json
import os
import shutil
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import random

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

def load_data(patient_id, window_size):
    data_path = f'/content/drive/My Drive/Msc Project/features_and_labels_detection/P{patient_id} features and seizure labels with window_{window_size}.csv'
    data = pd.read_csv(data_path)
    data = data.dropna()
    importance = pd.read_csv(f'/content/drive/My Drive/Msc Project/importance_feature/P{patient_id}MDI_importance with window_{window_size}.csv')
    top_features_list = importance.sort_values(by='Importance', ascending=False)['Unnamed: 0'].tolist()
    return data, top_features_list

def save_best_params(patient_id, window_size, best_params):
    if not os.path.exists(f'/content/drive/My Drive/Msc Project/best_params_mlp'):
        os.makedirs(f'/content/drive/My Drive/Msc Project/best_params_mlp')

    params_file = f'/content/drive/My Drive/Msc Project/best_params_mlp/params_patient_{patient_id}_window_{window_size}.json'
    with open(params_file, 'w') as file:
        json.dump(best_params, file)

def load_best_params(patient_id, window_size):
    params_file = f'/content/drive/My Drive/Msc Project/best_params_mlp/params_patient_{patient_id}_window_{window_size}.json'
    if os.path.exists(params_file):
        with open(params_file, 'r') as file:
            best_params = json.load(file)
        return best_params
    else:
        return None

def save_best_scores(patient_id, window_size, best_scores):
    if not os.path.exists(f'/content/drive/My Drive/Msc Project/best_scores_mlp'):
        os.makedirs(f'/content/drive/My Drive/Msc Project/best_scores_mlp')
    scores_file = f'/content/drive/My Drive/Msc Project/best_scores_mlp/best_scores.csv'
    row_name = f'{patient_id}_window_{window_size}'
    scores_df = pd.DataFrame(best_scores, index=[row_name])
    if os.path.exists(scores_file):
        existing_df = pd.read_csv(scores_file, index_col=0)
        if row_name in existing_df.index:
            existing_df.loc[row_name] = scores_df.loc[row_name]
            existing_df.to_csv(scores_file)
        else:
            updated_df = pd.concat([existing_df, scores_df])
            updated_df.to_csv(scores_file)
    else:
        scores_df.to_csv(scores_file)

def process_patient_window(patient_id, window_size):
    data, top_features_list = load_data(patient_id, window_size)

    selected_features = pd.read_csv(f'/content/drive/My Drive/Msc Project/select_feature/P{patient_id}_selected_features_with_window_{window_size}.csv')
    number_features = selected_features.loc[selected_features['AUC'].idxmax()]['Num Features']
    print(f'Number of features for patient {patient_id} with window {window_size}: {number_features}')

    X = data.drop(columns=['label', 'timestamp', 'seizure_event', 'Unnamed: 0'])
    X = X[top_features_list[:int(number_features)]]
    y = data['label']

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)
    smote = SMOTE(random_state=42, n_jobs=-1)
    X_train, y_train = smote.fit_resample(X_train, y_train)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)

    def create_model(trial):
        model = Sequential()
        model.add(Dense(trial.suggest_int('units1', 32, 256), activation='relu', input_shape=(X_train.shape[1],)))
        model.add(Dropout(trial.suggest_float('dropout1', 0.1, 0.5)))
        model.add(Dense(trial.suggest_int('units2', 32, 128), activation='relu'))
        model.add(Dropout(trial.suggest_float('dropout2', 0.1, 0.5)))
        model.add(Dense(trial.suggest_int('units3', 32, 64), activation='relu'))
        model.add(Dropout(trial.suggest_float('dropout3', 0.1, 0.5)))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=trial.suggest_float('lr', 1e-5, 1e-2, log=True)),
                      loss='binary_crossentropy',
                      metrics=['AUC'])
        return model

    def objective(trial):
        model = create_model(trial)
        checkpoint_path = f'/content/drive/My Drive/Msc Project/temp_weights/best_weights_trial_{trial.number}.h5'
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, monitor='val_auc', mode='max', save_weights_only=True)
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_auc', mode='max', patience=5, restore_best_weights=True)
        history = model.fit(X_train, y_train, epochs=50, shuffle=True, batch_size=trial.suggest_int('batch_size', 32, 128), verbose=1, validation_data=(X_val, y_val), callbacks=[checkpoint_callback, early_stopping])
        auc_val = max(history.history['val_auc'])

        # Save best checkpoint path in trial attributes
        best_epoch = np.argmax(history.history['val_auc']) + 1
        trial.set_user_attr("best_epoch", best_epoch)
        trial.set_user_attr("best_checkpoint_path", checkpoint_path)

        return auc_val

    class EarlyStoppingCallback:
        def __init__(self, patience: int):
           self.patience = patience
           self.best_value = None
           self.no_improvement_count = 0

        def __call__(self, study, trial):
           if self.best_value is None or study.best_value > self.best_value:
            self.best_value = study.best_value
            self.no_improvement_count = 0
           else:
            self.no_improvement_count += 1
           if self.no_improvement_count >= self.patience:
            study.stop()

    # Define the study
    study = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler(seed=SEED))

    # Load best parameters if available and enqueue them
    best_params = load_best_params(patient_id, window_size)
    if best_params:
        study.enqueue_trial(best_params)

    early_stopping = EarlyStoppingCallback(patience=10)
    # Optimize the objective function
    study.optimize(objective, n_trials=30,callbacks=[early_stopping])

    # Retrieve the best parameters and best checkpoint path
    best_params = study.best_params
    best_epoch = study.best_trial.user_attrs["best_epoch"]
    best_checkpoint_path = study.best_trial.user_attrs["best_checkpoint_path"]
    print("Best parameters found by Optuna:", best_params)
    print("Best epoch found by Optuna:", best_epoch)
    print("Best checkpoint path found by Optuna:", best_checkpoint_path)

    # Save the best parameters
    save_best_params(patient_id, window_size, best_params)

    # Save the best model weights to a new path
    best_weights_path = f'/content/drive/My Drive/Msc Project/best_weight_mlp/best_model_weights_patient_{patient_id}_window_{window_size}.h5'
    shutil.copy(best_checkpoint_path, best_weights_path)
    print("Best weights saved to:", best_weights_path)

    # Load the best weights for the final model
    best_model = create_model(optuna.trial.FixedTrial(best_params))
    best_model.load_weights(best_weights_path)

    # Evaluate the final model
    y_pred_val = (best_model.predict(X_val) > 0.5).astype("int32")
    y_pred_test = (best_model.predict(X_test) > 0.5).astype("int32")

    # Calculate ROC AUC scores  Note：（the sklearn build-in auc is slightly different from the keras build-in auc, use sklearn auc to keep consistence with other method that used)
    y_pred_val_scores = best_model.predict(X_val)
    auc_val = roc_auc_score(y_val, y_pred_val_scores)
    y_pred_test_scores = best_model.predict(X_test)
    auc_test = roc_auc_score(y_test, y_pred_test_scores)

    # Print classification reports and accuracy
    print(f"Validation classification report for patient {patient_id} with window {window_size}:")
    print(classification_report(y_val, y_pred_val))

    print(f"Test classification report for patient {patient_id} with window {window_size}:")
    print(classification_report(y_test, y_pred_test))

    print(f"Validation accuracy for patient {patient_id} with window {window_size}: {accuracy_score(y_val, y_pred_val)}")
    print(f"Test accuracy for patient {patient_id} with window {window_size}: {accuracy_score(y_test, y_pred_test)}")

    print(f"Validation AUC for patient {patient_id} with window {window_size}: {auc_val}")
    print(f"Test AUC for patient {patient_id} with window {window_size}: {auc_test}")

    best_scores = {
        "sensitivity": classification_report(y_test, y_pred_test, output_dict=True)['1']['recall'],
        "auc": auc_test
    }

    save_best_scores(patient_id, window_size, best_scores)

# List of patient IDs and window sizes '1_','3_','4_','10','13','19','23','27','29','30','31'
patient_ids = ['1_']
window_sizes = [24]

# Loop through each patient and each window size
for patient_id in patient_ids:
    for window_size in window_sizes:
        print(f"Processing patient {patient_id} with window size {window_size}")
        process_patient_window(patient_id, window_size)


Processing patient 1_ with window size 24


/usr/local/lib/python3.10/dist-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
[I 2024-07-17 01:09:12,706] A new study created in memory with name: no-name-247c506c-7795-48e8-a5ff-307a9857c95b


Number of features for patient 1_ with window 24: 14.0
Epoch 1/50
1936/1936 [==============================] - 9s 4ms/step - loss: 0.6870 - auc: 0.5774 - val_loss: 0.6189 - val_auc: 0.7764
Epoch 2/50
1936/1936 [==============================] - 7s 4ms/step - loss: 0.6405 - auc: 0.6739 - val_loss: 0.5983 - val_auc: 0.7904
Epoch 3/50
1936/1936 [==============================] - 7s 4ms/step - loss: 0.6203 - auc: 0.6997 - val_loss: 0.5884 - val_auc: 0.7983
Epoch 4/50
1936/1936 [==============================] - 7s 4ms/step - loss: 0.6081 - auc: 0.7157 - val_loss: 0.5843 - val_auc: 0.8016
Epoch 5/50
1936/1936 [==============================] - 7s 4ms/step - loss: 0.5996 - auc: 0.7253 - val_loss: 0.5835 - val_auc: 0.8025
Epoch 6/50
1936/1936 [==============================] - 7s 4ms/step - loss: 0.5938 - auc: 0.7323 - val_loss: 0.5739 - val_auc: 0.8012
Epoch 7/50
1936/1936 [==============================] - 7s 4ms/step - loss: 0.5893 - auc: 0.7370 - val_loss: 0.5712 - val_auc: 0.7997
Epoch 8

[I 2024-07-17 01:10:24,512] Trial 0 finished with value: 0.802548348903656 and parameters: {'units1': 116, 'dropout1': 0.4802857225639665, 'units2': 103, 'dropout2': 0.3394633936788146, 'units3': 37, 'dropout3': 0.16239780813448107, 'lr': 1.493656855461762e-05, 'batch_size': 116}. Best is trial 0 with value: 0.802548348903656.


Epoch 1/50
4583/4583 [==============================] - 19s 4ms/step - loss: 0.6412 - auc: 0.6677 - val_loss: 0.5494 - val_auc: 0.7934
Epoch 2/50
4583/4583 [==============================] - 16s 4ms/step - loss: 0.5950 - auc: 0.7324 - val_loss: 0.5482 - val_auc: 0.7927
Epoch 3/50
4583/4583 [==============================] - 16s 4ms/step - loss: 0.5815 - auc: 0.7484 - val_loss: 0.5378 - val_auc: 0.7905
Epoch 4/50
4583/4583 [==============================] - 16s 4ms/step - loss: 0.5718 - auc: 0.7585 - val_loss: 0.5404 - val_auc: 0.7844
Epoch 5/50
4583/4583 [==============================] - 17s 4ms/step - loss: 0.5622 - auc: 0.7684 - val_loss: 0.5313 - val_auc: 0.7852
Epoch 6/50
4583/4583 [==============================] - 16s 4ms/step - loss: 0.5534 - auc: 0.7774 - val_loss: 0.5114 - val_auc: 0.7858


...

Epoch 1/50
5223/5223 [==============================] - 21s 4ms/step - loss: 0.6210 - auc: 0.6938 - val_loss: 0.5982 - val_auc: 0.7783
Epoch 2/50
5223/5223 [==============================] - 18s 4ms/step - loss: 0.5846 - auc: 0.7397 - val_loss: 0.6135 - val_auc: 0.7766
Epoch 3/50
5223/5223 [==============================] - 18s 4ms/step - loss: 0.5702 - auc: 0.7556 - val_loss: 0.6027 - val_auc: 0.7748
Epoch 4/50
5223/5223 [==============================] - 19s 4ms/step - loss: 0.5592 - auc: 0.7676 - val_loss: 0.6264 - val_auc: 0.7794
Epoch 5/50
5223/5223 [==============================] - 19s 4ms/step - loss: 0.5496 - auc: 0.7773 - val_loss: 0.6402 - val_auc: 0.7862
Epoch 6/50
5223/5223 [==============================] - 18s 4ms/step - loss: 0.5416 - auc: 0.7855 - val_loss: 0.6170 - val_auc: 0.7785
Epoch 7/50
5223/5223 [==============================] - 18s 4ms/step - loss: 0.5328 - auc: 0.7942 - val_loss: 0.6496 - val_auc: 0.7796
Epoch 8/50
5223/5223 [==============================] -

[I 2024-07-17 01:25:03,888] Trial 9 finished with value: 0.7861920595169067 and parameters: {'units1': 33, 'dropout1': 0.42618457138193366, 'units2': 100, 'dropout2': 0.3916028672163949, 'units3': 57, 'dropout3': 0.12961786069363615, 'lr': 0.0001189589673755355, 'batch_size': 43}. Best is trial 0 with value: 0.802548348903656.


Epoch 1/50
1783/1783 [==============================] - 9s 4ms/step - loss: 0.6400 - auc: 0.6806 - val_loss: 0.5955 - val_auc: 0.7947
Epoch 2/50
1783/1783 [==============================] - 6s 4ms/step - loss: 0.5894 - auc: 0.7409 - val_loss: 0.5700 - val_auc: 0.7941
Epoch 3/50
1783/1783 [==============================] - 6s 4ms/step - loss: 0.5716 - auc: 0.7595 - val_loss: 0.5537 - val_auc: 0.7909
Epoch 4/50
1783/1783 [==============================] - 6s 4ms/step - loss: 0.5600 - auc: 0.7717 - val_loss: 0.5493 - val_auc: 0.7888
Epoch 5/50
1783/1783 [==============================] - 6s 3ms/step - loss: 0.5498 - auc: 0.7829 - val_loss: 0.5356 - val_auc: 0.7899
Epoch 6/50
1783/1783 [==============================] - 6s 4ms/step - loss: 0.5403 - auc: 0.7922 - val_loss: 0.5184 - val_auc: 0.7901


[I 2024-07-17 01:25:44,219] Trial 10 finished with value: 0.7947431802749634 and parameters: {'units1': 233, 'dropout1': 0.10239887318232685, 'units2': 99, 'dropout2': 0.10718475024592788, 'units3': 32, 'dropout3': 0.27523237098473197, 'lr': 1.1102804974349515e-05, 'batch_size': 126}. Best is trial 0 with value: 0.802548348903656.


Best parameters found by Optuna: {'units1': 116, 'dropout1': 0.4802857225639665, 'units2': 103, 'dropout2': 0.3394633936788146, 'units3': 37, 'dropout3': 0.16239780813448107, 'lr': 1.493656855461762e-05, 'batch_size': 116}
Best epoch found by Optuna: 5
Best checkpoint path found by Optuna: /content/drive/My Drive/Msc Project/temp_weights/best_weights_trial_0.h5
Best weights saved to: /content/drive/My Drive/Msc Project/best_weight_mlp/best_model_weights_patient_1__window_24.h5
447/447 [==============================] - 1s 1ms/step
Validation classification report for patient 1_ with window 24:
              precision    recall  f1-score   support

           0       1.00      0.62      0.76     13993
           1       0.05      0.90      0.09       288

    accuracy                           0.62     14281
   macro avg       0.52      0.76      0.42     14281
weighted avg       0.98      0.62      0.75     14281

Test classification report for patient 1_ with window 24:
              

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score
import optuna
import json
import os
import shutil
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import random

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

def load_data(patient_id, window_size,forecasting_length):
    data_path = f'/content/drive/My Drive/Msc Project/features_and_labels_detection/P{patient_id} features and seizure labels with window_{window_size}.csv'
    data = pd.read_csv(data_path)
    data = data.dropna()
    importance = pd.read_csv(f'/content/drive/My Drive/Msc Project/Forecasting/importance_feature/P{patient_id}MDI_importance with window_{window_size} with forecasting_{forecasting_length}.csv')
    top_features_list = importance.sort_values(by='Importance', ascending=False)['Unnamed: 0'].tolist()
    return data, top_features_list

def save_best_params(patient_id, window_size,forecasting_length, best_params):
    if not os.path.exists(f'/content/drive/My Drive/Msc Project/Forecasting/best_params_mlp'):
        os.makedirs(f'/content/drive/My Drive/Msc Project/Forecasting/best_params_mlp')

    params_file = f'/content/drive/My Drive/Msc Project/Forecasting/best_params_mlp/params_patient_{patient_id}_window_{window_size}_forecasting_{forecasting_length}.json'
    with open(params_file, 'w') as file:
        json.dump(best_params, file)

def load_best_params(patient_id, window_size,forecasting_length):
    params_file = f'/content/drive/My Drive/Msc Project/Forecasting/best_params_mlp/params_patient_{patient_id}_window_{window_size}_forecasting_{forecasting_length}.json'
    if os.path.exists(params_file):
        with open(params_file, 'r') as file:
            best_params = json.load(file)
        return best_params
    else:
        return None

def save_best_scores(patient_id, window_size,forecasting_length, best_scores):
    if not os.path.exists(f'/content/drive/My Drive/Msc Project/Forecasting/best_scores_mlp'):
        os.makedirs(f'/content/drive/My Drive/Msc Project/Forecasting/best_scores_mlp')
    scores_file = f'/content/drive/My Drive/Msc Project/Forecasting/best_scores_mlp/best_scores.csv'
    row_name = f'{patient_id}_window_{window_size}_forecasting_{forecasting_length}'
    scores_df = pd.DataFrame(best_scores, index=[row_name])
    if os.path.exists(scores_file):
        existing_df = pd.read_csv(scores_file, index_col=0)
        if row_name in existing_df.index:
            existing_df.loc[row_name] = scores_df.loc[row_name]
            existing_df.to_csv(scores_file)
        else:
            updated_df = pd.concat([existing_df, scores_df])
            updated_df.to_csv(scores_file)
    else:
        scores_df.to_csv(scores_file)

def process_patient_window(patient_id, window_size,forecasting_length):
    data, top_features_list = load_data(patient_id, window_size, forecasting_length)

    selected_features = pd.read_csv(f'/content/drive/My Drive/Msc Project/Forecasting/select_feature/P{patient_id}_selected_features_with_window_{window_size}_with_forecasting_{forecasting_length}.csv')
    number_features = selected_features.loc[selected_features['AUC'].idxmax()]['Num Features']
    print(f'Number of features for patient {patient_id} with window {window_size}: {number_features}')

    X = data.drop(columns=['label', 'timestamp', 'seizure_event', 'Unnamed: 0'])
    X = X[top_features_list[:int(number_features)]]
    y = data['label']
    y = y.shift(-forecasting_length)

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)
    smote = SMOTE(random_state=42, n_jobs=-1)
    X_train, y_train = smote.fit_resample(X_train, y_train)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)
    print(len(X_test))
    print(len(y_test))
    X_test = X_test.iloc[:-forecasting_length]
    y_test = y_test.iloc[:-forecasting_length]
    print(len(X_test))
    print(len(y_test))

    def create_model(trial):
        model = Sequential()
        model.add(Dense(trial.suggest_int('units1', 32, 256), activation='relu', input_shape=(X_train.shape[1],)))
        model.add(Dropout(trial.suggest_float('dropout1', 0.1, 0.5)))
        model.add(Dense(trial.suggest_int('units2', 32, 128), activation='relu'))
        model.add(Dropout(trial.suggest_float('dropout2', 0.1, 0.5)))
        model.add(Dense(trial.suggest_int('units3', 32, 64), activation='relu'))
        model.add(Dropout(trial.suggest_float('dropout3', 0.1, 0.5)))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=trial.suggest_float('lr', 1e-5, 1e-2, log=True)),
                      loss='binary_crossentropy',
                      metrics=['AUC'])
        return model

    def objective(trial):
        model = create_model(trial)
        checkpoint_path = f'/content/drive/My Drive/Msc Project/Forecasting/temp_weights/best_weights_trial_{trial.number}.h5'
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, monitor='val_auc', mode='max', save_weights_only=True)
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_auc', mode='max', patience=5, restore_best_weights=True)
        history = model.fit(X_train, y_train, epochs=50, shuffle=True, batch_size=trial.suggest_int('batch_size', 32, 128), verbose=1, validation_data=(X_val, y_val), callbacks=[checkpoint_callback, early_stopping])
        auc_val = max(history.history['val_auc'])

        # Save best checkpoint path in trial attributes
        best_epoch = np.argmax(history.history['val_auc']) + 1
        trial.set_user_attr("best_epoch", best_epoch)
        trial.set_user_attr("best_checkpoint_path", checkpoint_path)

        return auc_val

    class EarlyStoppingCallback:
        def __init__(self, patience: int):
           self.patience = patience
           self.best_value = None
           self.no_improvement_count = 0

        def __call__(self, study, trial):
           if self.best_value is None or study.best_value > self.best_value:
            self.best_value = study.best_value
            self.no_improvement_count = 0
           else:
            self.no_improvement_count += 1
           if self.no_improvement_count >= self.patience:
            study.stop()

    # Define the study
    study = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler(seed=SEED))

    # Load best parameters if available and enqueue them
    best_params = load_best_params(patient_id, window_size,forecasting_length)
    if best_params:
        study.enqueue_trial(best_params)

    early_stopping = EarlyStoppingCallback(patience=10)
    # Optimize the objective function
    study.optimize(objective, n_trials=30,callbacks=[early_stopping])

    # Retrieve the best parameters and best checkpoint path
    best_params = study.best_params
    best_epoch = study.best_trial.user_attrs["best_epoch"]
    best_checkpoint_path = study.best_trial.user_attrs["best_checkpoint_path"]
    print("Best parameters found by Optuna:", best_params)
    print("Best epoch found by Optuna:", best_epoch)
    print("Best checkpoint path found by Optuna:", best_checkpoint_path)

    # Save the best parameters
    save_best_params(patient_id, window_size,forecasting_length, best_params)

    # Save the best model weights to a new path
    best_weights_path = f'/content/drive/My Drive/Msc Project/Forecasting/best_weight_mlp/best_model_weights_patient_{patient_id}_window_{window_size}_forecasting_{forecasting_length}.h5'
    shutil.copy(best_checkpoint_path, best_weights_path)
    print("Best weights saved to:", best_weights_path)

    # Load the best weights for the final model
    best_model = create_model(optuna.trial.FixedTrial(best_params))
    best_model.load_weights(best_weights_path)

    # Evaluate the final model
    y_pred_val = (best_model.predict(X_val) > 0.5).astype("int32")
    y_pred_test = (best_model.predict(X_test) > 0.5).astype("int32")

    # Calculate ROC AUC scores  Note：（the sklearn build-in auc is slightly different from the keras build-in auc, use sklearn auc to keep consistence with other method that used)
    y_pred_val_scores = best_model.predict(X_val)
    auc_val = roc_auc_score(y_val, y_pred_val_scores)
    y_pred_test_scores = best_model.predict(X_test)
    auc_test = roc_auc_score(y_test, y_pred_test_scores)

    # Print classification reports and accuracy
    print(f"Validation classification report for patient {patient_id} with window {window_size}:")
    print(classification_report(y_val, y_pred_val))

    print(f"Test classification report for patient {patient_id} with window {window_size}:")
    print(classification_report(y_test, y_pred_test))

    print(f"Validation accuracy for patient {patient_id} with window {window_size}: {accuracy_score(y_val, y_pred_val)}")
    print(f"Test accuracy for patient {patient_id} with window {window_size}: {accuracy_score(y_test, y_pred_test)}")

    print(f"Validation AUC for patient {patient_id} with window {window_size}: {auc_val}")
    print(f"Test AUC for patient {patient_id} with window {window_size}: {auc_test}")

    best_scores = {
        "auc": auc_test,
        "sensitivity": classification_report(y_test, y_pred_test, output_dict=True)['1.0']['recall'],
        "specificity": classification_report(y_test, y_pred_test, output_dict=True)['0.0']['recall'],
        "accuracy": accuracy_score(y_test, y_pred_test)
    }

    save_best_scores(patient_id, window_size,forecasting_length, best_scores)

# List of patient IDs and window sizes '1_','3_','4_','10','13','19','23','27','29','30','31'
patient_ids = ['1_','4_','10','13','19','23','27','29','30','31']
window_sizes = [288]
forecasting_lengths = [24,120]

# Loop through each patient and each window size
for patient_id in patient_ids:
    for window_size in window_sizes:
        for forecasting_length in forecasting_lengths:
         print(f"Processing patient {patient_id} with window size {window_size} with forecasting length {forecasting_length}")
         process_patient_window(patient_id, window_size, forecasting_length)



Processing patient 1_ with window size 288 with forecasting length 24


/usr/local/lib/python3.10/dist-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
[I 2024-07-20 13:32:14,418] A new study created in memory with name: no-name-83266b77-1112-4433-9105-e21212a7027c


Number of features for patient 1_ with window 288: 1.0
14255
14255
14231
14231
Epoch 1/50
1647/1647 [==============================] - 11s 4ms/step - loss: 0.6702 - auc: 0.6420 - val_loss: 0.6947 - val_auc: 0.8090
Epoch 2/50
1647/1647 [==============================] - 6s 3ms/step - loss: 0.6548 - auc: 0.6536 - val_loss: 0.6598 - val_auc: 0.8094
Epoch 3/50
1647/1647 [==============================] - 6s 3ms/step - loss: 0.6516 - auc: 0.6544 - val_loss: 0.6393 - val_auc: 0.8091
Epoch 4/50
1647/1647 [==============================] - 6s 3ms/step - loss: 0.6507 - auc: 0.6548 - val_loss: 0.6271 - val_auc: 0.8091
Epoch 5/50
1647/1647 [==============================] - 6s 4ms/step - loss: 0.6495 - auc: 0.6555 - val_loss: 0.6187 - val_auc: 0.8095
Epoch 6/50
1647/1647 [==============================] - 6s 4ms/step - loss: 0.6486 - auc: 0.6559 - val_loss: 0.6142 - val_auc: 0.8082
Epoch 7/50
1647/1647 [==============================] - 6s 3ms/step - loss: 0.6475 - auc: 0.6561 - val_loss: 0.6071 

[I 2024-07-20 13:33:18,183] Trial 0 finished with value: 0.8094785809516907 and parameters: {'units1': 116, 'dropout1': 0.4802857225639665, 'units2': 103, 'dropout2': 0.3394633936788146, 'units3': 37, 'dropout3': 0.16239780813448107, 'lr': 1.493656855461762e-05, 'batch_size': 116}. Best is trial 0 with value: 0.8094785809516907.


Epoch 1/50
3899/3899 [==============================] - 16s 4ms/step - loss: 0.6606 - auc: 0.6498 - val_loss: 0.6397 - val_auc: 0.8095
Epoch 2/50
3899/3899 [==============================] - 13s 3ms/step - loss: 0.6495 - auc: 0.6559 - val_loss: 0.6144 - val_auc: 0.8082
Epoch 3/50
3899/3899 [==============================] - 13s 3ms/step - loss: 0.6471 - auc: 0.6557 - val_loss: 0.6065 - val_auc: 0.8047
Epoch 4/50
3899/3899 [==============================] - 13s 3ms/step - loss: 0.6449 - auc: 0.6578 - val_loss: 0.6015 - val_auc: 0.7920
Epoch 5/50
3899/3899 [==============================] - 13s 3ms/step - loss: 0.6427 - auc: 0.6584 - val_loss: 0.5992 - val_auc: 0.7844
Epoch 6/50
3899/3899 [==============================] - 13s 3ms/step - loss: 0.6400 - auc: 0.6588 - val_loss: 0.5976 - val_auc: 0.7761


...

Epoch 1/50
1849/1849 [==============================] - 9s 4ms/step - loss: 0.5934 - auc: 0.7418 - val_loss: 0.6158 - val_auc: 0.6400
Epoch 2/50
1849/1849 [==============================] - 7s 4ms/step - loss: 0.5119 - auc: 0.8246 - val_loss: 0.7056 - val_auc: 0.5900
Epoch 3/50
1849/1849 [==============================] - 7s 4ms/step - loss: 0.4430 - auc: 0.8743 - val_loss: 0.7667 - val_auc: 0.6033
Epoch 4/50
1849/1849 [==============================] - 7s 4ms/step - loss: 0.3879 - auc: 0.9057 - val_loss: 0.8533 - val_auc: 0.5979
Epoch 5/50
1849/1849 [==============================] - 7s 4ms/step - loss: 0.3478 - auc: 0.9250 - val_loss: 0.9368 - val_auc: 0.5919
Epoch 6/50
1849/1849 [==============================] - 7s 4ms/step - loss: 0.3105 - auc: 0.9406 - val_loss: 1.0135 - val_auc: 0.5847


[I 2024-07-20 23:46:26,007] Trial 13 finished with value: 0.6400455236434937 and parameters: {'units1': 165, 'dropout1': 0.11343435397378882, 'units2': 85, 'dropout2': 0.3579178144028043, 'units3': 48, 'dropout3': 0.28113660735026025, 'lr': 0.0002677178390304144, 'batch_size': 88}. Best is trial 4 with value: 0.6568694710731506.


Epoch 1/50
1535/1535 [==============================] - 8s 4ms/step - loss: 0.5860 - auc: 0.7506 - val_loss: 0.6213 - val_auc: 0.6298
Epoch 2/50
1535/1535 [==============================] - 5s 4ms/step - loss: 0.4906 - auc: 0.8411 - val_loss: 0.6976 - val_auc: 0.6192
Epoch 3/50
1535/1535 [==============================] - 6s 4ms/step - loss: 0.4265 - auc: 0.8846 - val_loss: 0.7413 - val_auc: 0.6296
Epoch 4/50
1535/1535 [==============================] - 6s 4ms/step - loss: 0.3927 - auc: 0.9037 - val_loss: 0.7553 - val_auc: 0.6371
Epoch 5/50
1535/1535 [==============================] - 6s 4ms/step - loss: 0.3674 - auc: 0.9164 - val_loss: 0.8326 - val_auc: 0.6147
Epoch 6/50
1535/1535 [==============================] - 5s 4ms/step - loss: 0.3471 - auc: 0.9261 - val_loss: 0.8443 - val_auc: 0.6166
Epoch 7/50
1535/1535 [==============================] - 6s 4ms/step - loss: 0.3328 - auc: 0.9322 - val_loss: 0.8582 - val_auc: 0.6069
Epoch 8/50
1535/1535 [==============================] - 6s 4ms

[I 2024-07-20 23:47:18,560] Trial 14 finished with value: 0.6370927095413208 and parameters: {'units1': 210, 'dropout1': 0.4985986522319813, 'units2': 115, 'dropout2': 0.4989975280446522, 'units3': 64, 'dropout3': 0.4193402045949063, 'lr': 0.0016706223345449034, 'batch_size': 106}. Best is trial 4 with value: 0.6568694710731506.


Best parameters found by Optuna: {'units1': 46, 'dropout1': 0.4795542149013333, 'units2': 125, 'dropout2': 0.4233589392465845, 'units3': 42, 'dropout3': 0.13906884560255356, 'lr': 0.0011290133559092666, 'batch_size': 74}
Best epoch found by Optuna: 1
Best checkpoint path found by Optuna: /content/drive/My Drive/Msc Project/Forecasting/temp_weights/best_weights_trial_4.h5
Best weights saved to: /content/drive/My Drive/Msc Project/Forecasting/best_weight_mlp/best_model_weights_patient_31_window_288_forecasting_120.h5
441/441 [==============================] - 1s 1ms/step
Validation classification report for patient 31 with window 288:
              precision    recall  f1-score   support

         0.0       0.73      0.88      0.80      9571
         1.0       0.57      0.34      0.43      4654

    accuracy                           0.70     14225
   macro avg       0.65      0.61      0.61     14225
weighted avg       0.68      0.70      0.68     14225

Test classification report for p